# Conversational Text Chatbot with Gemini and LangChain for Farmers

In this notebook you will use Gemini and LangChain to build a conversational text-based chatbot

## Install Gemini and LangChain dependencies

Google Gemini API is free (till now). You can get a key [here](https://aistudio.google.com/app/apikey), just need to sign in with your google account. Gemini may not be available fully in EU.

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-google-genai==2.0.7
!pip install langchain-community==0.3.11
!pip install pyngrok==7.2.2
!pip install PyMuPDF==1.24.0
!pip install chromadb==0.6.3
!pip install pydantic==2.10.1
!pip install langchain-chroma==0.2.2
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.24
    Uninstalling langsmith-0.3.24:
      Successfully uninstalled langsmith-0.3.24
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy

## Install RAG Evaluation Libraries

In [ ]:
!pip install ragas==0.2.3
!pip install deepeval==1.4.7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: ragas
    Found existing installation: ragas 0.2.14
    Uninstalling ragas-0.2.14:
      Successfully uninstalled ragas-0.2.14


## Load Gemini API Credentials

Here we load it from a secret keys of Colab so we don't expose the credentials on the internet by mistake

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

## Load Necessary Dependencies and Gemini LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", convert_system_message_to_human=True)

## Build a Conversational Text Chatbot App

In [ ]:
# Import necessary components from the LangChain library.
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [ ]:
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain.schema.runnable.config import RunnableConfig
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import StrOutputParser
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from operator import itemgetter
import chromadb
import tempfile
import os
import pandas as pd

In [ ]:
docs = []
docs_path = "/content/"

if not os.path.exists(docs_path):
    os.makedirs(docs_path)

for file in os.listdir(docs_path):
    if file.endswith(".pdf"):
        # print(file)
        loader = PyMuPDFLoader(os.path.join(docs_path, file))
        #loader = CSVLoader(file_path=os.path.join(docs_path, file))
        docs.extend(loader.load())




data = loader.load()

print(data)

if docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                                 chunk_overlap=200)
    chunks = text_splitter.split_documents(docs)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    client = chromadb.PersistentClient(path="./chroma_db")
    # Use Chroma from LangChain
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        client=client,
        collection_name="document_collection"
    )

    # Define retriever object
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})

/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 7 of document /content/farmerbook-1-20.pdf
  warnings.warn(


[Document(metadata={'source': '/content/farmerbook-1-20.pdf', 'file_path': '/content/farmerbook-1-20.pdf', 'page': 0, 'total_pages': 20, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20250408072807Z', 'trapped': ''}, page_content='A holistic  \nperspective of  \nscientific  \nagriculture\nA joint initiative to  \nimpart farmers with \ntechnical knowledge on \nbasic agriculture.\nFarmer’s Handbook on Basic Agriculture\n'), Document(metadata={'source': '/content/farmerbook-1-20.pdf', 'file_path': '/content/farmerbook-1-20.pdf', 'page': 1, 'total_pages': 20, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20250408072807Z', 'trapped': ''}, page_content='Disclaimer:\nThe opinions expressed provided in this publication are those of the authors and do not necessarily reflect those of  G

In [ ]:
def run_agribrain_chatbot(system_prompt='You are a Agricultural Expert', history_window=30,
                        temperature=0.3, llm=gemini_model):
  # Modify the core behavior of the LLM
  if system_prompt:
    SYS_PROMPT = system_prompt
  else:
    SYS_PROMPT = """
                  Act as a helpful AI Assistant for farmers. Use the following context information to answer questions.
                 """
  def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

  # Function to format retrieved documents
  #def format_docs(docs):
    # First, print the retrieved documents in a well-defined format
    #print("\n" + "="*50)
    #print("📚 RETRIEVED CONTENT:")
    #print("="*50)

    #for i, doc in enumerate(docs, 1):
      #print(f"\n📄 Document {i}:")
      #print("-"*40)
      #print(doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content)
      #if hasattr(doc, 'metadata') and doc.metadata:
        #print("\n📋 Metadata:")
        #for key, value in doc.metadata.items():
          #print(f"  • {key}: {value}")

    #print("\n" + "="*50 + "\n")

    # Then return the formatted content as before
    #return "\n\n".join(doc.page_content for doc in docs)

  # Create a prompt template to store conversation history and accept new user prompts
  # Combine system prompt and context into a single system message for Gemini
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", "{system_prompt}\n\nContext information: {context}"),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{question}"),
    ]
  )

  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=history_window,
                                          return_messages=True)

  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
      system_prompt=lambda x: SYS_PROMPT,
      history=lambda x: memory.load_memory_variables({})["history"],
      question=lambda x: x["question"]
    )
    |
    prompt
    |
    llm
  )

  # Print a welcome message when the chatbot starts.
  print("🌾 Hello! I'm AgriBrain, your agri-assistant. Ask me anything about crops, soil, or farming. (type 'STOP' to quit)")

  # Start an infinite loop for interactive conversation with the user.
  while True:
    # Get input from the user.
    user_prompt = input('👨‍🌾 You: >>> ')
    # Check if the user wants to end the chat.
    if user_prompt.strip().upper() == 'STOP':
      print("👋 AgriBrain: >>> Take care and happy farming!")
      break

    # Generate and print the chatbot's reply.
    user_inp = {'question': user_prompt}
    reply = conversation_chain.invoke(user_inp)
    print(f"🌱 Agribrain: >>>\n{reply.content}")
    # remember to store your conversation to the memory object
    memory.save_context({"input": user_prompt}, {"output": reply.content})

In [ ]:
run_agribrain_chatbot()

<ipython-input-6-75e76f69f245>:45: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=history_window,


🌾 Hello! I'm AgriBrain, your agri-assistant. Ask me anything about crops, soil, or farming. (type 'STOP' to quit)
👨‍🌾 You: >>> What are the benefits of Mulching?


<ipython-input-6-75e76f69f245>:51: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🌱 Agribrain: >>>
Based on the provided text, the benefits of crop residue mulching are:

*   **Increased availability of water and organic matter:** Mulch helps retain moisture in the soil and as it decomposes, it adds organic matter, improving soil fertility.
*   **Less erosion:** Mulch protects the soil surface from the impact of raindrops and wind, reducing soil erosion.
*   **Environment protection:** By reducing erosion and improving soil health, mulching contributes to environmental protection.
*   **Less drought susceptibility:** Improved water retention in the soil makes crops less vulnerable to drought.
*   **Improved soil quality and fertilizer efficiency:** Mulch enhances soil structure, water infiltration, and nutrient availability, leading to better fertilizer utilization.
*   **Minimizes long-term dependency on external inputs:** By improving soil health and fertility, mulching reduces the need for synthetic fertilizers and other external inputs.
👨‍🌾 You: >>> what kind of

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🌱 Agribrain: >>>
Based on the symptoms you described (yellowing and curling of leaf blades, restricted ear production, poor grain set, and indeterminate tillering in cereals), the deficiency most likely to cause these issues is **Nitrogen (N)**.

Here's why:

*   **Yellowing (Chlorosis):** Nitrogen is a key component of chlorophyll, the pigment responsible for the green color in plants. A deficiency leads to reduced chlorophyll production, resulting in yellowing, particularly in older leaves first.
*   **Restricted Ear Production and Poor Grain Set:** Nitrogen is crucial for overall plant growth and development, including reproductive processes. Insufficient nitrogen can hinder ear development and grain filling.
*   **Indeterminate Tillering:** While other deficiencies can sometimes affect tillering, nitrogen deficiency can lead to excessive, weak tillering as the plant tries to compensate for its lack of resources.
*   **Curling of Leaf Blades:** While not always present, leaf curling

# RAG Evaluation

## Create a vector storage and a retriver

In [ ]:
docs = []
docs_path = "/content/"

if not os.path.exists(docs_path):
    os.makedirs(docs_path)

for file in os.listdir(docs_path):
    if file.endswith(".pdf"):
        # print(file)
        loader = PyMuPDFLoader(os.path.join(docs_path, file))
        docs.extend(loader.load())

if docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                                 chunk_overlap=200)
    chunks = text_splitter.split_documents(docs)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    client = chromadb.PersistentClient(path="./chroma_db")
    # Use Chroma from LangChain
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        client=client,
        collection_name="document_collection"
    )

    # Define retriever object
    retriever = vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.3})

/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 7 of document /content/farmerbook-1-20.pdf
  warnings.warn(


## Create the RAG pipeline

In [ ]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content))
        print()

In [ ]:
query = "what is Agriculture?"
top_docs = retriever.invoke(query)
display_docs(top_docs)

Metadata: {'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20250408072807Z', 'page': 5, 'producer': 'iLovePDF', 'source': '/content/farmerbook-1-20.pdf', 'subject': '', 'title': '', 'total_pages': 20, 'trapped': ''}
Content Brief:


Farmer’s Handbook on Basic Agriculture
Preface
A
griculture is an important sector of Indian Economy as more than half of its population relies on Ag-
riculture as principle source of income.  Research and Extension systems play major role in generation 
and dissemination of Agricultural technologies aiming at enhancing the income of farmers. The extension 
system adopts series of extension methods such as Training, demonstration, exposure visit to transfer 
the technologies from lab to land. Majority of these extension efforts mainly focus on location and crop 
specific technologies, and mostly on solution to problem basis. However, there is a need for equipping the 
farmers with Basic knowledge of Agriculture in order to create a better knowledge platform at farmer level 
for taking appropriate farm management decisions and to absorb modern technologies.  
In view of this, Desai Fruits and Vegetables Pvt. Ltd. (DFV), India, in cooperation with the Deutsche 
Gesellschaft für Internationale Zusammenarbeit (GIZ) GmbH on behalf of the German Federal Ministry 
for Economic Cooperation and Development (BMZ) in close cooperation with National Institute of Ag-
ricultural Extension Management (MANAGE- An Organization of Ministry of Agriculture, Government 
of India) brought out Farmer’s Handbook on Basic Agriculture to impart technical knowledge on Basic 
Agriculture to farmers to provide holistic perspective of scientific Agriculture.


Metadata: {'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20250408072807Z', 'page': 4, 'producer': 'iLovePDF', 'source': '/content/farmerbook-1-20.pdf', 'subject': '', 'title': '', 'total_pages': 20, 'trapped': ''}
Content Brief:


Farmer’s Handbook on Basic Agriculture
Acknowledgement
H
igher demand for agricultural raw material is now anticipated and agriculture is not any more about 
producing farm products and selling them exclusively at the local market. Instead farmers today have 
a world market to serve. But the new chances bring new challenges. Farmers and agricultural enterprises, 
willing to be part of the new expanding world market, not only have to take into consideration customers’ 
preferences whom they want to serve, but also adhere to international trade regulations set by WTO and 
comply with high production and quality standards required by the importing countries.
Agriculture contributes around 17% to GDP and continues to be among the most important and success-
ful sectors in India. Around 58% of the Indian population depend on agriculture for their livelihood. Apart 
from delivering the local industries with top quality raw materials for processing, agriculture provides 
almost 10% of total export earnings. However, to support the impressive Indian economic growth in the 
coming years, agriculture will have to contribute more towards value addition, productivity enhancement, 
high quality products and trained manpower to successfully tackle these challenges.
The states of Gujarat and Maharashtra have competitive advantages for the production of several com-
modities. However, productivity and competitiveness remains low. Rising quality requirements of export


Metadata: {'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20250408072807Z', 'page': 5, 'producer': 'iLovePDF', 'source': '/content/farmerbook-1-20.pdf', 'subject': '', 'title': '', 'total_pages': 20, 'trapped': ''}
Content Brief:


soil fertility in an economically and environmentally sustainable manner. It also focuses on the need for 
soil testing, plant nutrition requirement, organic & inorganic fertilizers, and Integrated Nutrient Manage-
ment (INM) for efficient, economic and sustainable production of crops.
The third chapter of the book is about Pest Management, and focuses on enhancing the awareness of and 
understanding among farmers about the crop pests, diseases and weed management through Integrated 
Pest Management. It also aims at sensitizing farmers on safe handling of chemicals and plant protection 
equipments as also elaborated further in the fifth chapter on “Occupational health and safety of farmers”
. 
It creates awareness about causes, preventive measures of health hazards, risks & fatalities in agriculture, 
and use of first aid in emergencies. It further includes safety tips and care to reduce the risk of injuries and 
fatalities while handling machineries and pesticides by farmers.
Time and resources management is an integral part of each and every activity, be it service sector, busi-
ness or day-to-day activities of life. Farming sector too has not remained untouched by it. Therefore, the 
fourth chapter of the book is devoted to “Farm Management”
. It is to educate and equip the farmers to 
make proper plans, take appropriate decisions and also to take advantage of the improved technologies

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer to the point based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    gemini_model
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [ ]:
import pprint

query = "What is Agriculture?"
result = rag_chain_w_sources.invoke(query)
pprint.pprint(result)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'context': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20250408072807Z', 'page': 5, 'producer': 'iLovePDF', 'source': '/content/farmerbook-1-20.pdf', 'subject': '', 'title': '', 'total_pages': 20, 'trapped': ''}, page_content='Farmer’s Handbook on Basic Agriculture\nPreface\nA\ngriculture is an important sector of Indian Economy as more than half of its population relies on Ag-\nriculture as principle source of income.  Research and Extension systems play major role in generation \nand dissemination of Agricultural technologies aiming at enhancing the income of farmers. The extension \nsystem adopts series of extension methods such as Training, demonstration, exposure visit to transfer \nthe technologies from lab to land. Majority of these extension efforts mainly focus on location and crop \nspecific technologies, and mostly on solution to problem basis. However, t

# Create End-to-End RAG Evaluation Workflow

![](https://i.imgur.com/GUIkpjy.png)

## Create a Synthetic RAG Golden Reference Dataset

In [ ]:
from langchain.docstore.document import Document
processed_docs = []

for doc in docs:
    # metadata = {
    #     "title": doc['title'],
    #     "id": doc['id'],
    # }
    data = doc.page_content
    processed_docs.append(Document(page_content=data)) #, metadata=metadata
processed_docs[:3]

[Document(metadata={}, page_content='A holistic  \nperspective of  \nscientific  \nagriculture\nA joint initiative to  \nimpart farmers with \ntechnical knowledge on \nbasic agriculture.\nFarmer’s Handbook on Basic Agriculture\n'),
 Document(metadata={}, page_content='Disclaimer:\nThe opinions expressed provided in this publication are those of the authors and do not necessarily reflect those of  GIZ . The \ndesignations employed and the presentation of material in this publication do not imply the expression of any opinion what-\nsoever on the part of  GIZ  concerning the legal status of any country, territory, city or area, or concerning the delimitation of \nits frontiers or boundaries.\n'),
 Document(metadata={}, page_content=' \nFarmer’s Handbook on Basic Agriculture\nFarmer’s Handbook on Basic Agriculture\nPrepared & compiled by\nDr. P. Chandra Shekara\nNational Institute of Agricultural Extension  \nManagement (MANAGE)\nMinistry of Agriculture, GoI\nHyderabad, Andhra Pradesh\nIn

In [ ]:
doc_contexts = [doc.page_content for doc in processed_docs]
doc_contexts[:3]

['A holistic  \nperspective of  \nscientific  \nagriculture\nA joint initiative to  \nimpart farmers with \ntechnical knowledge on \nbasic agriculture.\nFarmer’s Handbook on Basic Agriculture\n',
 'Disclaimer:\nThe opinions expressed provided in this publication are those of the authors and do not necessarily reflect those of  GIZ . The \ndesignations employed and the presentation of material in this publication do not imply the expression of any opinion what-\nsoever on the part of  GIZ  concerning the legal status of any country, territory, city or area, or concerning the delimitation of \nits frontiers or boundaries.\n',
 ' \nFarmer’s Handbook on Basic Agriculture\nFarmer’s Handbook on Basic Agriculture\nPrepared & compiled by\nDr. P. Chandra Shekara\nNational Institute of Agricultural Extension  \nManagement (MANAGE)\nMinistry of Agriculture, GoI\nHyderabad, Andhra Pradesh\nIndia\nDr. N. Balasubramani\nNational Institute of Agricultural Extension  \nManagement (MANAGE)\nMinistry of

In [ ]:
from deepeval.synthesizer import Synthesizer
from deepeval.synthesizer import types

/usr/local/lib/python3.11/dist-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.7, however version 2.7.0 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
print("hi")

hi


In [ ]:
len(doc_contexts)

20

In [ ]:
synthesizer = Synthesizer(model='gpt-4o', embedder=embeddings) # model='gemini-2.0-flash',

eval_data = synthesizer.generate_goldens(
    # Provide a list of context for synthetic data generation
    contexts=[[doc] for doc in doc_contexts],
    include_expected_output=True,
    max_goldens_per_context=1,
    num_evolutions=1,
    scenario="Retrieval Augmented Generation",
    task="Question Answering",
    evolutions={
        types.Evolution.REASONING: 0.1,     # Evolves the input to require multi-step logical thinking.
        types.Evolution.MULTICONTEXT: 0.9,  # Ensures that all relevant information from the context is utilized.
        types.Evolution.CONCRETIZING: 0.0,  # Makes abstract ideas more concrete and detailed.
        types.Evolution.CONSTRAINED: 0.0,   # Introduces a condition or restriction, testing the model's ability to operate within specific limits.
        types.Evolution.COMPARATIVE: 0.0,   # Requires a response that involves a comparison between options or contexts.
        types.Evolution.HYPOTHETICAL: 0.0,  # Forces the model to consider and respond to a hypothetical scenario.
        types.Evolution.IN_BREADTH: 0.0,    # Broadens the input to touch on related or adjacent topics.
    }
)

Event loop is already running. Applying nest_asyncio patch to allow async execution...


✨ Generating up to 20 goldens using DeepEval (using gpt-4o, use case=QA, method=default): 100%|██████████| 20/20 [00:28<00:00,  1.43s/it]


✓ Generation finished 🎉! You can also run 'deepeval login' to generate and save goldens directly on Confident AI.

## Save the Synthetic RAG Golden Reference Dataset

In [ ]:
import dill

In [ ]:
with open('golden_ref_data.bin', 'wb') as f:
    dill.dump(eval_data, f)

## Create RAG Evaluation Dataset

In [ ]:
from deepeval.dataset import EvaluationDataset

eval_dataset = EvaluationDataset()

# load golden dataset
with open('golden_ref_data.bin', 'rb') as f:
    golden_docs = dill.load(f)

eval_dataset.goldens = golden_docs

In [ ]:
eval_dataset.goldens[0]

Golden(input='How do contour trenching and straw thatching help protect seedlings from frost?', actual_output=None, expected_output='Contour trenching helps in runoff collection, which indirectly aids in maintaining soil moisture and moderating temperature fluctuations, offering some protection against frost. Straw thatching provides direct protection for young seedlings against cold by covering them, thereby helping to mitigate frost impact.', context=['General Conditions for Cultivation of Crops\nFarmer’s Handbook on Basic Agriculture\n9\nCoping options for farmers continued....\n \nContour trenching for runoff  collec-\ntion\nConventional Raised Bed Planting\n• \n20-25% Saving in irrigation water\n \nShelterbelts\n• \nShelterbelts reduce wind velocity.\n• \nModerate temperature.\n• \nReduce evaporative loss and conserve soil \nmoisture.\nStraw Thatching\n• \nProtecting young seedlings against cold by cov-\nering with straw thatching.\nFrost Protection\n'], retrieval_context=None, ad

In [ ]:
eval_dataset.goldens[0].input

'How do contour trenching and straw thatching help protect seedlings from frost?'

In [ ]:
rag_chain_w_sources.invoke(eval_dataset.goldens[0].input)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'context': [Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': 'D:20250408072807Z', 'page': 16, 'producer': 'iLovePDF', 'source': '/content/farmerbook-1-20.pdf', 'subject': '', 'title': '', 'total_pages': 20, 'trapped': ''}, page_content='General Conditions for Cultivation of Crops\nFarmer’s Handbook on Basic Agriculture\n9\nCoping options for farmers continued....\n \nContour trenching for runoff  collec-\ntion\nConventional Raised Bed Planting\n• \n20-25% Saving in irrigation water\n \nShelterbelts\n• \nShelterbelts reduce wind velocity.\n• \nModerate temperature.\n• \nReduce evaporative loss and conserve soil \nmoisture.\nStraw Thatching\n• \nProtecting young seedlings against cold by cov-\nering with straw thatching.\nFrost Protection'),
  Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '/content/farmerbook-1-20.pdf', 'format': 'PDF 1.5', '

In [ ]:
from typing import List
from deepeval.test_case import LLMTestCase
from deepeval.dataset import Golden
from tqdm import tqdm
import time
import random

def convert_goldens_to_test_cases(goldens: List[Golden]) -> List[LLMTestCase]:
    test_cases = []
    # Calculate sleep time to respect RPM limit
    # For 15 RPM, we need to wait at least 4 seconds between calls
    # Adding some jitter to avoid exact timing patterns
    base_sleep_time = 60 / 15  # 4 seconds

    for i, golden in enumerate(tqdm(goldens)):
        # Make the API call
        response_obj = rag_chain_w_sources.invoke(golden.input)

        # Create test case
        test_case = LLMTestCase(
            input=golden.input,
            actual_output=response_obj['response'],
            expected_output=golden.expected_output,
            context=golden.context,
            retrieval_context=[doc.page_content for doc in response_obj['context']]
        )
        test_cases.append(test_case)

        # Sleep to respect rate limits, but only if not the last item
        if i < len(goldens) - 1:
            # Add some jitter (±15%) to avoid predictable patterns
            jitter = random.uniform(0.85, 1.15)
            sleep_time = base_sleep_time * jitter
            time.sleep(sleep_time)

    return test_cases

In [ ]:
eval_dataset.test_cases = convert_goldens_to_test_cases(eval_dataset.goldens)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
  5%|▌         | 1/20 [00:05<01:37,  5.11s/it]/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
 10%|█         | 2/20 [00:09<01:23,  4.62s/it]/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
 15%|█▌        | 3/20 [00:14<01:23,  4.90s/it]/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Co

In [ ]:
eval_dataset.test_cases[0]

LLMTestCase(input='How do contour trenching and straw thatching help protect seedlings from frost?', actual_output="Straw thatching protects young seedlings against the cold. I'm sorry, but the text does not describe how contour trenching helps protect seedlings from frost.\n", expected_output='Contour trenching helps in runoff collection, which indirectly aids in maintaining soil moisture and moderating temperature fluctuations, offering some protection against frost. Straw thatching provides direct protection for young seedlings against cold by covering them, thereby helping to mitigate frost impact.', context=['General Conditions for Cultivation of Crops\nFarmer’s Handbook on Basic Agriculture\n9\nCoping options for farmers continued....\n \nContour trenching for runoff  collec-\ntion\nConventional Raised Bed Planting\n• \n20-25% Saving in irrigation water\n \nShelterbelts\n• \nShelterbelts reduce wind velocity.\n• \nModerate temperature.\n• \nReduce evaporative loss and conserve so

## Run and View RAG Evaluations on the Evaluation Dataset

In [ ]:
from deepeval import evaluate
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, HallucinationMetric
from deepeval.metrics.ragas import RAGASAnswerRelevancyMetric

contextual_precision = ContextualPrecisionMetric(threshold=0.5, include_reason=True, model="gpt-4o")
contextual_recall = ContextualRecallMetric(threshold=0.5, include_reason=True, model="gpt-4o")
contextual_relevancy = ContextualRelevancyMetric(threshold=0.5, include_reason=True, model="gpt-4o")
answer_relevancy = AnswerRelevancyMetric(threshold=0.5, include_reason=True, model="gpt-4o")
faithfulness = FaithfulnessMetric(threshold=0.5, include_reason=True, model="gpt-4o")
hallucination = HallucinationMetric(threshold=0.5, include_reason=True, model="gpt-4o")
ragas_answer_relevancy = RAGASAnswerRelevancyMetric(threshold=0.5, embeddings=embeddings, model="gpt-4o")

eval_results = evaluate(test_cases=eval_dataset.test_cases,
                        metrics=[contextual_precision, contextual_recall, contextual_relevancy,
                                 answer_relevancy, ragas_answer_relevancy, faithfulness, hallucination])

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy (ragas) Metric! (using gpt-4o, strict=False, 
async_mode=True)...

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

None


Evaluating 20 test case(s) in parallel: |██████████|100% (20/20) [Time Taken: 01:23,  4.19s/test case]



Metrics Summary

  - ✅ Contextual Precision (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because all nodes are relevant and appropriately ranked. The nodes in the retrieval context extensively address how the Farmer's Handbook enhances farmers' decision-making abilities, highlighting aspects like 'enhancing awareness among farmers about sources of extension, information and services,' alignment with key topics such as 'Soil and Plant Nutrition,' and providing 'a holistic perspective of scientific Agriculture.' Great job!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because all sentences in the expected output align perfectly with nodes in the retrieval context, demonstrating an excellent match. Great job!, error: None)
  - ✅ Contextual Relevancy (score: 0.8823529411764706, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [ ]:
eval_results.test_results[0]

TestResult(success=True, metrics_data=[MetricData(name='Contextual Precision', threshold=0.5, success=True, score=1.0, reason="The score is 1.00 because all nodes are relevant and appropriately ranked. The nodes in the retrieval context extensively address how the Farmer's Handbook enhances farmers' decision-making abilities, highlighting aspects like 'enhancing awareness among farmers about sources of extension, information and services,' alignment with key topics such as 'Soil and Plant Nutrition,' and providing 'a holistic perspective of scientific Agriculture.' Great job!", strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.00764, verbose_logs='Verdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": "The text provides insights about enhancing farmers\' decision-making capabilities, mentioning \'educate and equip the farmers to make proper plans, take appropriate decisions\' and the last chapter focusing on \'enhancing awareness among farmers about

In [ ]:
eval_metrics = []
for result in eval_results.test_results:
    eval_dict = {}
    eval_dict['Input'] = result.input
    eval_dict['Expected Output'] = result.expected_output
    eval_dict['Actual Output'] = result.actual_output
    eval_dict['Context'] = result.context
    eval_dict['Retrieval Context'] = result.retrieval_context
    eval_dict['Success'] = result.success
    metrics = result.metrics_data
    for metric in metrics:
        eval_dict[metric.name+'_Score'] = metric.score
    for metric in metrics:
        eval_dict[metric.name+'_Success'] = metric.success
    for metric in metrics:
        eval_dict[metric.name+'_Reason'] = metric.reason
    eval_metrics.append(eval_dict)

In [ ]:
eval_metrics[0]

{'Input': "How can the chapters in the Farmer’s Handbook enhance farmers' ability to make informed agricultural decisions?",
 'Expected Output': "The chapters in the Farmer’s Handbook enhance farmers' ability to make informed agricultural decisions by providing comprehensive knowledge and practical guidance on key aspects of farming. The first chapter equips farmers with foundational knowledge of Good Agricultural Practices and resource management, enabling them to select appropriate crops and cropping patterns. The second chapter on soil and plant nutrition helps farmers understand soil properties and nutrient management, encouraging sustainable and efficient crop production. The third chapter focuses on pest management, educating farmers on handling pests and diseases safely. The fourth chapter educates farmers on farm management strategies, enabling them to plan and use resources effectively. Lastly, the final chapter enhances farmers' access to information and services, including a

In [ ]:
import pandas as pd

eval_results_df = pd.DataFrame(eval_metrics)
eval_results_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Input,How can the chapters in the Farmer’s Handbook ...,What topics are addressed in the 'Farmer’s Han...,What vital guidelines from the Farmer's Handbo...,How do contour trenching and straw thatching h...,In what ways does PFI enhance the decision-mak...,"In what ways do climatic shifts, such as tempe...","How do climatic, soil, water, economic, and te...",What insights does the Farmer's Handbook provi...,What is the impact of rills and unattended gul...,What are the effects of increased temperatures...,In what ways do Farm Ponds and Conservation Fu...,How does farmers' access to agri-tech knowledg...,"What role do monsoons, temperature, and market...",What is GIZ's stance on authors' opinions and ...,What topics are covered in the 'Soil and Plant...,"In what ways can altering cropping, employing ...","Who are the contributors from GIZ, universitie...",How does Retrieval Augmented Generation effect...,What are the benefits of zero tillage farming ...,"What are the roles of runoff, different types ..."
Expected Output,The chapters in the Farmer’s Handbook enhance ...,The 'Farmer’s Handbook on Basic Agriculture' b...,"For effective crop cultivation, key guidelines...","Contour trenching helps in runoff collection, ...",PFI enhances the decision-making and competiti...,Climatic shifts like temperature spikes and ra...,"Climatic, soil, water, economic, and technolog...",The Farmer's Handbook on Basic Agriculture pro...,Rills can develop into larger gullies if left ...,Increased temperatures and altered rainfall in...,Farm Ponds and Conservation Furrows contribute...,Farmers' access to agri-tech knowledge and pra...,"Monsoons, temperature, and market influences s...",GIZ does not necessarily reflect the opinions ...,The 'Soil and Plant Nutrition' section of the ...,"Altering cropping patterns, employing mixed fa...","The contributors from GIZ are Mr. Max Baumann,...",Retrieval Augmented Generation (RAG) enhances ...,Zero tillage farming offers several benefits f...,"Runoff, erosion, and vegetation play crucial r..."
Actual Output,The Farmer's Handbook chapters aim to educate ...,The 'Farmer’s Handbook on Basic Agriculture' b...,"Based on the Farmer's Handbook, vital guidelin...",Straw thatching protects young seedlings again...,PFI enables farmers to be self-sufficient deci...,"Climatic shifts, such as temperature spikes an...","Climatic factors such as temperature, rainfall...",The Farmer's Handbook on Basic Agriculture imp...,"Rills, if neglected, can grow into large gulli...",Increased temperatures may lead to an increase...,Farm Ponds and Conservation Furrows contribute...,Farmers with access to technical knowledge on ...,Monsoons are a key source of water in agricult...,"According to the disclaimer, the opinions expr...",The 'Soil and Plant Nutrition' section (pp. 33...,Changing cropping patterns and mixed cropping ...,"The contributors from GIZ are Max Baumann, Fre...","I am sorry, but this document does not contain...",Zero tillage farming has several benefits:\n\n...,Here's what the provided text says about the r...
Context,[ \nFarmer’s Handbook on Basic Agriculture\nPr...,[ \nFarmer’s Handbook on Basic Agriculture\nFa...,[2\nGeneral Conditions for Cultivation of Crop...,[General Conditions for Cultivation of Crops\n...,[Farmer’s Handbook on Basic Agriculture\nAckno...,[General Conditions for Cultivation of Crops\n...,[General Conditions for Cultivation of Crops\n...,[A holistic \nperspective of \nscientific \...,[General Conditions for Cultivation of Crops\n...,[6\nGeneral Conditions for Cultivation of Crop...,[8\nGeneral Conditions for Cultivation of Crop...,[General Conditions for Cultivation of Crops\n...,[4\nGeneral Conditions for Cultivation of Crop...,[Disclaimer:\nThe opinions expressed provided ...,[ \nFarmer’s Handbook on Basic Agriculture\nCo...,[General Conditions for Cultivation of Crops\n...,[The Authors acknowledge the contribution of f...,[],[12\nGeneral Co

In [ ]:
eval_results_df[['Contextual Precision_Score', 'Contextual Recall_Score', 'Contextual Relevancy_Score',
                 'Answer Relevancy_Score', 'Answer Relevancy (ragas)_Score',
                 'Faithfulness_Score', 'Hallucination_Score']].describe()

,Contextual Precision_Score,Contextual Recall_Score,Contextual Relevancy_Score,Answer Relevancy_Score,Answer Relevancy (ragas)_Score,Faithfulness_Score,Hallucination_Score
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.925000,0.874107,0.465945,0.950000,0.766760,0.956667,0.050000
std,0.226045,0.253559,0.296240,0.223607,0.186336,0.153364,0.223607
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000
25%,1.000000,0.842857,0.210714,1.000000,0.771348,1.000000,0.000000
50%,1.000000,1.000000,0.510870,1.000000,0.809074,1.000000,0.000000
75%,1.000000,1.000000,0.698584,1.000000,0.832060,1.000000,0.000000
max,1.000000,1.000000,0.882353,1.000000,0.901560,1.000000,1.000000
